In [2]:
from surprise import SVD
from surprise import Dataset
from surprise import accuracy
from surprise import Reader
from surprise.model_selection import train_test_split

import pandas as pd

In [3]:
tags = pd.read_table("tags.dat", encoding = "ISO-8859-1")
user_contacts_timestamps = pd.read_table("user_contacts-timestamps.dat", encoding = "ISO-8859-1")
user_contacts = pd.read_table("user_contacts.dat", encoding = "ISO-8859-1")
user_taggedbookmarks = pd.read_table("user_taggedbookmarks.dat", encoding = "ISO-8859-1")
user_taggedbookmarks_timestamps = pd.read_table("user_taggedbookmarks-timestamps.dat", encoding = "ISO-8859-1")
user_contacts = pd.read_table("user_contacts.dat", encoding = "ISO-8859-1")
bookmark_tags = pd.read_table("bookmark_tags.dat", encoding = "ISO-8859-1")
bookmarks = pd.read_table("bookmarks.dat", encoding = "ISO-8859-1")

In [ ]:
tags.head()

In [ ]:
user_contacts_timestamps.head()

In [ ]:
user_contacts.head()

In [18]:
user_taggedbookmarks.head()

,userID,bookmarkID,tagID,day,month,year,hour,minute,second
0,8,1,1,8,11,2010,23,29,22
1,8,2,1,8,11,2010,23,25,59
2,8,7,1,8,11,2010,18,55,1
3,8,7,6,8,11,2010,18,55,1
4,8,7,7,8,11,2010,18,55,1


In [ ]:
user_taggedbookmarks_timestamps.head()

In [ ]:
user_contacts.head()

In [ ]:
bookmarks.head(100)

In [4]:
data = user_taggedbookmarks.groupby(['userID','bookmarkID'],as_index=False)['tagID'].count().rename(columns={'tagID': 'Counts'})
data.head()

,userID,bookmarkID,Counts
0,8,1,1
1,8,2,1
2,8,7,3
3,8,8,3
4,8,9,2


In [5]:
import implicit
import numpy as np
import scipy.sparse as sparse

In [6]:
users = list(np.sort(data.userID.unique()))
emits = list(np.sort(data.bookmarkID.unique()))
counts = list(data.Counts)

In [7]:
rows = data.userID.astype(int)
cols = data.bookmarkID.astype(int)

In [15]:
data_sparse = sparse.csr_matrix((counts, (cols, rows)), shape=(108036, 108036))

In [16]:
from implicit.als import AlternatingLeastSquares
model = AlternatingLeastSquares(factors=50)
model.fit(data_sparse)

100%|██████████| 15.0/15 [00:01<00:00, 10.22it/s]


In [22]:
userid = 8

user_items = data_sparse.T.tocsr()
recommendations = model.recommend(userid, user_items)

for r in recommendations:
    print(list(bookmarks[bookmarks['id']==r[0]]['url'])[0])

http://www.voki.com/
http://www.sldirectory.com/
http://www.youtube.com/watch?v=-dnL00TdmLY
http://www.search-cube.com/
http://www.visuwords.com/
http://udleditions.cast.org/index.html
http://www.gutenberg.org/wiki/Main_Page
http://www.teachertube.com/
http://blachan.com/shahi/
http://animoto.com/
